In [1]:
import time
import requests
from bs4 import BeautifulSoup

In [2]:
response = requests.get('http://abehiroshi.la.coocan.jp/top.htm')
time.sleep(1)
print(response.text)


<html>

<head>
<meta http-equiv="Content-Type" content="text/html; charset=Shift_JIS">
<meta http-equiv="Content-Style-Type" content="text/css">
<meta name="GENERATOR" content="JustSystems Homepage Builder Version 20.0.6.0 for Windows">
<title>¢ °Ìz[y[W</title>


</head>


<body background="image/abehiroshi.jpg">
<br>
<h1 align="center">¢ °Ìz[y[W</h1>

<div align="center">
<!--
  <table>
    <tbody>
      <tr>
        <td><b></b></td>
      </tr>
    </tbody>
  </table>
<br>
-->

</div>
<table align="center">
  <tr>
    <td rowspan="2"><img src="abe-top-20190328-2.jpg" width="350" height="414" border="0"><br>
      <br>
      <table width="256">
        <tr>
          <td width="14">&nbsp;</td>
          <td width="230">¢ °i × Ðëµj<br>
        Hiroshi Abe<br>
        </td>
        </tr>
        <tr>
          <td>&nbsp;</td>
          <td>¶Nú 1964N622ú<br></td>
        </tr>
        <tr>
          <td>&nbsp;</td>
          <td>t^ A^<br>

In [3]:
response.encoding = response.apparent_encoding or "shift_jis"   # 文字コードの指定
soup = BeautifulSoup(response.text, "html.parser")              # HTML のパース

title = soup.title.get_text(strip=True)                         # 1) タイトル
heading = soup.find("h1").get_text(strip=True)                  # 2) ページ中央の <h1>
main_img_src = soup.find("img")["src"]                          # 3) メイン画像（最初に出てくる <img>）

# 4) プロフィール表（<td rowspan="2"> 内の最初の <table>）
profile_table = soup.find("td", attrs={"rowspan": "2"}).find("table")       
rows = profile_table.find_all("tr")
name        = rows[0].find_all("td")[1].get_text(strip=True)
birthday    = rows[1].find_all("td")[1].get_text(strip=True)
affiliation = rows[2].find_all("td")[1].get_text(strip=True)
profile_url = rows[3].find("a")["href"]

# 5) 問い合わせメールアドレス
email_href = soup.select_one('a[href^="mailto:"]')["href"]
email = email_href.replace("mailto:", "")

# 6) 最新出演作（<a target="_blank"> を走査し “New!” の有無も判定）
works = []
for a in soup.select('a[target="_blank"]'):
    strong = a.find_parent("strong")
    if strong is None:
        continue

    title_txt = a.get_text(strip=True)
    rest_txt  = strong.get_text(strip=True).replace(title_txt, "")
    is_new    = strong.find("font", attrs={"color": "#ff0000"}) is not None

    works.append(
        {
            "title": title_txt,
            "url": a["href"],
            "published": rest_txt,
            "is_new": is_new,
        }
    )

# 結果表示
print("タイトル        :", title)
print("見出し          :", heading)
print("メイン画像 SRC  :", main_img_src)
print("\n--- プロフィール ---")
print("名前            :", name)
print("生年月日        :", birthday)
print("所属            :", affiliation)
print("詳細ページ URL  :", profile_url)
print("メールアドレス  :", email)
print("\n--- 最新出演作 ---")

for w in works:
    mark = "★New" if w["is_new"] else ""
    print(f"{w['title']} ({w['published']}) {mark} → {w['url']}")


タイトル        : 阿部 寛のホームページ
見出し          : 阿部 寛のホームページ
メイン画像 SRC  : abe-top-20190328-2.jpg

--- プロフィール ---
名前            : 阿部 寛（あべ ひろし）Hiroshi Abe
生年月日        : 生年月日 1964年6月22日
所属            : 血液型 A型
詳細ページ URL  : prof/prof.htm
メールアドレス  : info@office-a2023.co.jp

--- 最新出演作 ---
俺ではない炎上 (「」New!2025年9月26日公開) ★New → https://movies.shochiku.co.jp/oredehanai-enjo
キャンドルスティック (「」2025年7月4日公開)  → https://candlestick.jp/
